In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
cd drive/MyDrive/Research1_code/

In [ ]:
cd FashionMNIST/LeNet-5

In [4]:
import numpy as np
import sys, os, random
import matplotlib.pyplot as plt
import pickle, gzip
from tqdm import tqdm,tqdm_notebook
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
batch_size = 128
seed_num = 81

# For reproducibility when you run the file with .py
torch.cuda.is_available()
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True

torch.backends.cudnn.deterministic =True

In [6]:
# Data Augmentation 
train_transform = transforms.Compose([transforms.RandomRotation(30), transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.2860,],[0.3205,])])
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.2860,],[0.3205,])])

# Splitting the training and test datasets
train_data = datasets.FashionMNIST(os.getcwd(), train=True,
                              download=True, transform=train_transform)
test_data = datasets.FashionMNIST(os.getcwd(), train=False,
                             download=True, transform=test_transform)

In [7]:
# Split the training set indices into training and validation set indices using 80:20 ratio
np.random.seed(seed_num)
len_trainset = len(train_data)
index_list = list(range(len_trainset))
np.random.shuffle(index_list)
split_index = 50000
train_indices, valid_indices =  index_list[:split_index], index_list[split_index:]

# Creating Samplers for training and validation set using the indices
np.random.seed(seed_num)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

torch.manual_seed(seed_num)

train_iterator = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_iterator = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [8]:
# LeNet-5 Model

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5), stride=1, padding=2, bias= True)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1, padding=0, bias= True)


        # Fully connected 1
        self.fc1 = nn.Linear(16*5*5, 120,bias= True) 

        # Fully connected 2
        self.fc2 = nn.Linear(120, 84,bias= True) 

        # Fully connected 3
        self.fc3 = nn.Linear(84,10, bias= True) 
    
    def forward(self, x):
        # Set 1
        out = F.max_pool2d(F.relu(self.cnn1(x)), kernel_size=(2, 2), stride=2)
        
        # Set 2
        out = F.max_pool2d(F.relu(self.cnn2(out)), kernel_size=(2, 2), stride=2) 

        
        # Flatten
        out = out.view(out.size(0), -1)

        # FC1
        out = F.relu(self.fc1(out))

        # FC2
        out = F.relu(self.fc2(out))

        # FC3
        out = self.fc3(out)
        
        return out


In [9]:
torch.manual_seed(seed_num)
unit=128

# Summary
model = LeNet()
print("Model:\n",model)

Model:
 LeNet(
  (cnn1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
# Layer names
layer_name = [n for n, p in model.named_parameters()]
layer_name

['cnn1.weight',
 'cnn1.bias',
 'cnn2.weight',
 'cnn2.bias',
 'fc1.weight',
 'fc1.bias',
 'fc2.weight',
 'fc2.bias',
 'fc3.weight',
 'fc3.bias']

In [11]:
# Loading the weights of ternary model 
model = torch.load('LeNet_5_Quant.pt')
model = model.cuda()
print("Loading weights done !")

# Total number of ternary weights (+w, -w)
totalParams = 0
for i in layer_name:
  print(i,(model.state_dict()[i] !=0).sum())
  totalParams +=  (model.state_dict()[i] !=0).sum()
    
print("Total Parameters:",totalParams, '\n')

Loading weights done !
cnn1.weight tensor(96, device='cuda:0')
cnn1.bias tensor(3, device='cuda:0')
cnn2.weight tensor(414, device='cuda:0')
cnn2.bias tensor(15, device='cuda:0')
fc1.weight tensor(616, device='cuda:0')
fc1.bias tensor(50, device='cuda:0')
fc2.weight tensor(784, device='cuda:0')
fc2.bias tensor(61, device='cuda:0')
fc3.weight tensor(475, device='cuda:0')
fc3.bias tensor(10, device='cuda:0')
Total Parameters: tensor(2524, device='cuda:0') 



In [12]:
# Model's performance on test set

correct_count, all_count = 0, 0
model.eval()
for images,labels in test_iterator:
      for image,label in zip(images,labels):

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Test Accuracy =", (correct_count/(all_count)))


Number Of Images Tested = 10000

Model Test Accuracy = 0.7421


In [13]:
# For each layer, model's ternary weights
state_dict = model.state_dict()

layer_distinct_weights = {}

for i in layer_name:
  imd = torch.unique(model.state_dict()[i])
  print(i+ ' hidden layer dimension', model.state_dict()[i].shape)
  print("Unique values of weight in "+ i+ " th hidden layer : ", imd)
  layer_distinct_weights[i] = imd.cpu().numpy().tolist()
  print()

cnn1.weight hidden layer dimension torch.Size([6, 1, 5, 5])
Unique values of weight in cnn1.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn1.bias hidden layer dimension torch.Size([6])
Unique values of weight in cnn1.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn2.weight hidden layer dimension torch.Size([16, 6, 5, 5])
Unique values of weight in cnn2.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

cnn2.bias hidden layer dimension torch.Size([16])
Unique values of weight in cnn2.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

fc1.weight hidden layer dimension torch.Size([120, 400])
Unique values of weight in fc1.weight th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

fc1.bias hidden layer dimension torch.Size([120])
Unique values of weight in fc1.bias th hidden layer :  tensor([-1.,  0.,  1.], device='cuda:0')

fc2.weight hidden layer dimension torch.Size([84, 120])
Unique values of weight i

In [14]:
keys = list(state_dict.keys())
print(keys)

['cnn1.bias', 'cnn1.weight', 'cnn2.bias', 'cnn2.weight', 'fc1.bias', 'fc1.weight', 'fc2.bias', 'fc2.weight', 'fc3.bias', 'fc3.weight']


In [15]:
for i in keys:
  print(i, state_dict[i].shape)

cnn1.bias torch.Size([6])
cnn1.weight torch.Size([6, 1, 5, 5])
cnn2.bias torch.Size([16])
cnn2.weight torch.Size([16, 6, 5, 5])
fc1.bias torch.Size([120])
fc1.weight torch.Size([120, 400])
fc2.bias torch.Size([84])
fc2.weight torch.Size([84, 120])
fc3.bias torch.Size([10])
fc3.weight torch.Size([10, 84])


In [16]:
# Duplicate architecture of the LeNet-5 Model

class LeNet1(nn.Module):
    def __init__(self, dn_info, dn_info1, dn_info2):
        super(LeNet1, self).__init__()

        self.dn_info = dn_info
        self.dn_info1 = dn_info1
        self.dn_info2 = dn_info2
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=(5,5), stride=1, padding=2, bias= True)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=(5,5), stride=1, padding=0, bias= True)


        # Fully connected 1
        self.fc1 = nn.Linear(16*5*5, 120,bias= True) 

        # Fully connected 2
        self.fc2 = nn.Linear(120, 84,bias= True) 

        # Fully connected 3
        self.fc3 = nn.Linear(84,10, bias= True) 
    
    def forward(self, x):
        # Set 1
        out = F.max_pool2d(F.relu(self.cnn1(x)), kernel_size=(2, 2), stride=2)
        
        # Set 2
        out = F.max_pool2d(F.relu(self.cnn2(out)), kernel_size=(2, 2), stride=2) 

        
        # Flatten
        out = out.view(out.size(0), -1)

        ####################################
        # Storing dead neurons indices
        idx = torch.where(out.cpu() == 0.)[1]
        
        for j in idx:
            self.dn_info[str(j.item())] += 1
            
        #####################################

        # FC1
        out = F.relu(self.fc1(out))

        ####################################
        # Storing dead neurons indices
        idx1 = torch.where(out.cpu() == 0.)[1]
        
        for j in idx1:
            self.dn_info1[str(j.item())] += 1
            
        #####################################

        # FC2
        out = F.relu(self.fc2(out))

        ####################################
        # Storing dead neurons indices
        idx2 = torch.where(out.cpu() == 0.)[1]
        
        for j in idx2:
            self.dn_info2[str(j.item())] += 1
            
        #####################################

        # FC3
        out = self.fc3(out)
        
        return out


In [17]:
def getDeadN_info(dn_info, dn_info1, dn_info2, unit, unit1, unit2, state_dict, ds, nameOftheSet):

  for i in range(unit):
      dn_info[str(i)] = 0

  for i in range(unit1):
    dn_info1[str(i)] = 0

  for i in range(unit2):
    dn_info2[str(i)] = 0
      
  model1 = LeNet1(dn_info, dn_info1, dn_info2)
  model1 = model1.cuda()

  model1.load_state_dict(state_dict)

  correct_count, all_count = 0, 0
  model1.eval()

  for images,labels in ds:
        for image,label in zip(images,labels):

          if torch.cuda.is_available():
              img = image.cuda()
              lab = label.cuda()
              img = img[None,].type('torch.cuda.FloatTensor')

          with torch.no_grad():
              output_ = model1(img) 

          pred_label = output_.argmax()

          if(pred_label.item()==lab.item()):
            correct_count += 1
          all_count += 1

  print("Number Of Images =", all_count)
  print(f"Model {nameOftheSet} Accuracy =", (correct_count/(all_count)))

In [18]:
dn_info_train = {}
dn_info_val = {}
dn_info_test = {}

dn_info_train1 = {}
dn_info_val1 = {}
dn_info_test1 = {}

dn_info_train2 = {}
dn_info_val2 = {}
dn_info_test2 = {}

getDeadN_info(dn_info= dn_info_train, dn_info1= dn_info_train1, dn_info2= dn_info_train2, unit = 800, unit1 = 120, unit2 = 84, state_dict = state_dict, ds= train_iterator, nameOftheSet = "Training")
getDeadN_info(dn_info= dn_info_val, dn_info1= dn_info_val1, dn_info2= dn_info_val2, unit = 800, unit1 = 120, unit2 = 84, state_dict = state_dict, ds= val_iterator, nameOftheSet = "Validation")
getDeadN_info(dn_info= dn_info_test, dn_info1= dn_info_test1, dn_info2= dn_info_test2, unit = 800, unit1 = 120, unit2 = 84, state_dict = state_dict, ds= test_iterator, nameOftheSet = "Test")

Number Of Images = 50000
Model Training Accuracy = 0.72974
Number Of Images = 10000
Model Validation Accuracy = 0.7268
Number Of Images = 10000
Model Test Accuracy = 0.7421


In [19]:
max_dn_val = max(dn_info_val.values())
max_dn_test = max(dn_info_test.values())
max_dn_train = max(dn_info_train.values())

max_dn_val1 = max(dn_info_val1.values())
max_dn_test1 = max(dn_info_test1.values())
max_dn_train1 = max(dn_info_train1.values())

max_dn_val2 = max(dn_info_val2.values())
max_dn_test2 = max(dn_info_test2.values())
max_dn_train2 = max(dn_info_train2.values())

max_dn_train, max_dn_val, max_dn_test, max_dn_train1, max_dn_val1, max_dn_test1, max_dn_train2, max_dn_val2, max_dn_test2

(48297, 9679, 9465, 50000, 10000, 10000, 50000, 10000, 10000)

In [20]:
dead_n_idx = [] 

for i, j in dn_info_train.items():
  if j == max_dn_train:
    dead_n_idx.append(i)

dead_n_idx1 = [] 

for i, j in dn_info_train1.items():
  if j == max_dn_train1:
    dead_n_idx1.append(i)

dead_n_idx2 = [] 

for i, j in dn_info_train2.items():
  if j == max_dn_train2:
    dead_n_idx2.append(i)

In [21]:
print("FC1 Layer :",len(dead_n_idx)) # Number of neurons that are dead
print("FC2 Layer :",len(dead_n_idx1)) # Number of neurons that are dead
print("FC3 Layer :",len(dead_n_idx2)) # Number of neurons that are dead

FC1 Layer : 1
FC2 Layer : 49
FC3 Layer : 13


In [22]:
print(dead_n_idx) # Indices of neuron that are dead

['120']


In [23]:
print(dead_n_idx1) # Indices of neuron that are dead

['1', '2', '4', '5', '7', '10', '11', '14', '17', '19', '20', '25', '26', '27', '32', '34', '36', '38', '42', '46', '48', '51', '52', '53', '55', '58', '60', '61', '65', '71', '72', '73', '74', '76', '81', '82', '85', '86', '89', '93', '96', '97', '99', '100', '104', '107', '113', '116', '118']


In [24]:
print(dead_n_idx2) # Indices of neuron that are dead

['2', '5', '21', '25', '57', '63', '64', '65', '69', '73', '75', '77', '83']


In [25]:
state_dict1 = state_dict

In [26]:
layer_distinct_weights

{'cnn1.bias': [-1.0, 0.0, 1.0],
 'cnn1.weight': [-1.0, 0.0, 1.0],
 'cnn2.bias': [-1.0, 0.0, 1.0],
 'cnn2.weight': [-1.0, 0.0, 1.0],
 'fc1.bias': [-1.0, 0.0, 1.0],
 'fc1.weight': [-1.0, 0.0, 1.0],
 'fc2.bias': [-1.0, 0.0, 1.0],
 'fc2.weight': [-1.0, 0.0, 1.0],
 'fc3.bias': [-1.0, 1.0],
 'fc3.weight': [-1.0, 0.0, 1.0]}

In [27]:
total_trans = 0
layers_name = ['fc1.weight', 'fc2.weight', 'fc3.weight']

n_idx = [dead_n_idx, dead_n_idx1, dead_n_idx2]

for ix,l in enumerate(layers_name):
  print(l)
  z = state_dict1[l]

  if len(layer_distinct_weights[l]) > 2 :
    w_neg, w_0, w_pos =  layer_distinct_weights[l]
    print(w_neg, w_0, w_pos)
  else:
    w_neg, w_pos = layer_distinct_weights[l]
    print(w_neg, w_pos)

  for idx in n_idx[ix]:
      if 'bias' in l:
        imd = z[eval(idx)]
      else:
        imd = z[:,eval(idx)]

      trans = torch.where(imd == w_neg)[0].nelement() + torch.where(imd == w_pos)[0].nelement()
      total_trans += trans *2
    
print("Total dead transitions for the layer :", total_trans)

fc1.weight
-1.0 0.0 1.0
fc2.weight
-1.0 0.0 1.0
fc3.weight
-1.0 0.0 1.0
Total dead transitions for the layer : 428


In [ ]:
# Fault coverage that we have obtained from main file : 4314 / 5048

In [28]:
# Adding more 428 transitions to the numerator will make the net fault coverage
(4314 + 428)/5048

0.9393819334389857